# 0. 전처리

In [10]:
import pymysql
import pandas as pd
from datetime import datetime, timedelta
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [11]:
from datetime import datetime, timedelta

today = str(datetime.today())[:10]
yesterday = str(datetime.today() - timedelta(1))[:10]

print(today, yesterday)

2022-08-02 2022-08-01


In [57]:
# conn = pymysql.connect(
#     user='my_srv',              
#     passwd='wkrldi@duqhdi12',   
#     host='125.141.223.156',     
#     db='m_yeoboya_ai_dv',            
#     charset='utf8',        
#     port=33141   
# )

# # member_msg_log_back
# cursor = conn.cursor(pymysql.cursors.DictCursor)
# sql = f"select * from m_yeoboya.member_msg_log_back where DATE(ins_date) between '{yesterday}' and '{today}'" 
# cursor.execute(sql)
# result = cursor.fetchall()
# cursor.close()
# msg_df = pd.DataFrame(result)
# msg_df

In [13]:
conn = pymysql.connect(
    user='my_srv',              
    passwd='wkrldi@duqhdi12',   
    host='125.141.223.156',     
    db='m_yeoboya_ai_dv',            
    charset='utf8',        
    port=33141   
)


cursor = conn.cursor(pymysql.cursors.DictCursor)
sql = "select * from m_yeoboya.member_mate" 
cursor.execute(sql)
result = cursor.fetchall()
cursor.close()
mate_df = pd.DataFrame(result) 
mate_df.head(3)

,mem_no,mem_sex,mem_loc,mem_l_code,mem_birth_year,mem_birth_month,mem_birth_ddi,mate_slct,mate_slct_yn,mem_name_slct,...,family_conts_upd_cnt,secret_item_yn,secret_yn,respect_person_cont,my_point_cont,list_prt_yn,my_hd_item_yn,my_hd_yn,ins_date,upd_date
0,5,m,q,qnz,1977,1,9,w,y,y,...,13,y,y,,90,y,n,n,2015-07-22 16:31:35,2022-07-29 09:00:59
1,1793293,m,c,c11,1997,9,5,w,y,y,...,0,n,n,,70,y,n,n,2022-03-05 00:10:45,2022-07-24 20:58:05
2,1897363,m,c,c01,1989,1,9,w,y,y,...,0,n,n,,40,y,n,n,2022-07-23 21:58:27,2022-07-23 21:58:27


In [14]:
# member_basic
cursor = conn.cursor(pymysql.cursors.DictCursor)
sql = "select * from m_yeoboya.member_basic" 
cursor.execute(sql)
result = cursor.fetchall()
cursor.close()
basic_df = pd.DataFrame(result)
basic_df.head(3)

,mem_no,mem_id,mem_passwd,mem_passwd_v1,mem_passwd_v2,passwd_tmp_yn,mem_nick,mem_nick_yn,mem_sex,mem_birth_year,...,mate_reg_yn,mem_slct,matchmk_mem_no,matchmk_conts,smtp_slct,smtp_id,app_ver,phone_ver,mem_chnl,upd_date
0,2,ag315@gmail.com,*EE8B70CCF4C155606422917955B852444BB120A9,d8f93948f3d3c283102c19e6db44356451b9985540178e...,,n,디노킹,y,m,1971,...,y,b,0,,b,BD31B3FA-7EE3-4739-A2BB-23B3D1A8BDC3,6.6.113,15.5,501,2015-07-22 16:21:21
1,5,dntn20@naver.com,*9CC98632AA58449C1C67E7A30BDC3647BFAA6293,f8dc94048d37ead74a70a0a94bdbc9e3fcdec63258c5c9...,,n,바쉐린,n,m,1977,...,y,b,0,,a,E7C0EFB3-D01F-4B77-AE67-05FD81B5C5F0,1.5.327,10,501,2015-07-22 16:25:49
2,57,baroom@naver.com,*BE52216085D67EE7619B0D23D61C04D14FBA8922,7c07727cb559b31f69ee693a92e4eeada400f13bc370cd...,,n,허니버터,y,m,1977,...,y,b,0,,a,b59e1c528cfb2268,1.5.323,12,501,2015-07-27 09:03:18


In [15]:
# member_pf_view_all_log
cursor = conn.cursor(pymysql.cursors.DictCursor)
sql = "select * from m_yeoboya.member_pf_view_all_log where ins_date between '2022-07-01' and '2022-07-02'" 
cursor.execute(sql)
result = cursor.fetchall()
cursor.close()
view_df = pd.DataFrame(result) 
view_df.head(3)

,auto_no,mem_no,mem_sex,ptr_mem_no,ptr_mem_sex,ptr_rprsn_photo_yn,open_slct,open_date,open_type,item_cnt,ins_date
0,341346362,1879332,f,240746,m,n,v,2022-07-01 00:00:00,0,0,2022-07-01 00:00:00
1,341346364,1879332,f,240746,m,n,f,2022-07-01 00:00:04,0,0,2022-07-01 00:00:04
2,341346366,689210,f,1229328,m,n,f,2022-07-01 00:00:04,0,0,2022-07-01 00:00:04


In [16]:
# 1개만 카운트하기 위해 중복 회원 쌍 제거, member_mate에 있는 회원만 사용
view_df.drop_duplicates(inplace=True, subset=['mem_no','ptr_mem_no'])
view_df.reset_index(drop=True,inplace=True)
view_df = view_df[view_df['mem_no'].isin(mate_df['mem_no'].values)]
view_df = view_df[view_df['ptr_mem_no'].isin(mate_df['mem_no'].values)]
print(len(view_df))

121553


In [17]:
# member_msg_log_back
cursor = conn.cursor(pymysql.cursors.DictCursor)
sql = "select * from m_yeoboya.member_msg_log_back where ins_date between '2022-07-01' and '2022-07-02'" 
cursor.execute(sql)
result = cursor.fetchall()
cursor.close()
msg_df = pd.DataFrame(result)
msg_df.head(3)

,auto_no,mem_no,mem_sex,mem_birth_year,mate_slct,login_media,mem_slct,ptr_mem_no,ptr_mem_sex,ptr_mem_birth_year,ptr_mate_slct,ptr_login_media,read_yn,msg_type,msg_slct,msg_cont,msg_etc,file_name,file_size,ins_date
0,211716802,956036,m,1980,w,s,o,1618323,f,1983,w,s,n,1,0,맘에드니깐 다시했쥬ㅠ,"{""mediaFileName"":"""",""mediaPlayTime"":"""",""mediaP...",,,2022-07-01 00:00:01
1,211716804,1233233,m,1970,r,x,n,1849564,f,1979,r,s,n,1,0,넘 매력 있으시고,,,,2022-07-01 00:00:03
2,211716806,663465,m,1980,w,s,o,1486107,f,1979,r,s,n,1,0,안녕하세요? 요즘같은 코로나 시국에 몸 건강히 하루하루 잘보내고 계시죠?^^ 메세지...,"{""mediaFileName"":"""",""mediaPlayTime"":"""",""mediaP...",,,2022-07-01 00:00:03


In [18]:
# 1개만 카운트하기 위해 중복 회원 쌍 제거, member_mate에 있는 회원만 사용
msg_df.drop_duplicates(inplace=True, subset=['mem_no','ptr_mem_no'])
msg_df = msg_df[msg_df['mem_no'].isin(mate_df['mem_no'].values)]
msg_df = msg_df[msg_df['ptr_mem_no'].isin(mate_df['mem_no'].values)]
msg_df.reset_index(drop=True,inplace=True)
len(msg_df)

19823

In [19]:
# age 연산
mate_df['age'] = int(str(datetime.today())[:4]) - mate_df['mem_birth_year']
df = mate_df[['mem_no','mem_sex','age']]

### 회원별 리스트/횟수 생성
- 열람 리스트/횟수, 피열람 리스트/횟수, 발신 리스트/횟수, 수신 리스트/횟수

In [20]:
# x가 열람한 회원 리스트
df['view_list'] = None
for i in tqdm(range(len(df))) :
    view_list = view_df[view_df['mem_no']==df.loc[i, 'mem_no']]['ptr_mem_no'].values.tolist()
    df.loc[i, 'view_list'] = [[view_list]]
    
# x를 열람한 회원 리스트
df['viewed_list'] = None
for i in tqdm(range(len(df))) :
    viewed_list = view_df[view_df['ptr_mem_no']==df.loc[i, 'mem_no']]['mem_no'].values.tolist()
    df.loc[i, 'viewed_list'] = [[viewed_list]]

100%|█████████████████████████████████████████████████████████████████████████| 124992/124992 [06:54<00:00, 301.19it/s]


In [21]:
df['view_list'] = df['view_list'].apply(lambda x : x[0])
df['view_count'] = df['view_list'].apply(lambda x : len(x))

df['viewed_list'] = df['viewed_list'].apply(lambda x : x[0])
df['viewed_count'] = df['viewed_list'].apply(lambda x : len(x))

In [22]:
# 메시지 발신 회원 리스트
df['send_list'] = None
for i in tqdm(range(len(df))) :
    send_list = msg_df[msg_df['mem_no']==df.loc[i, 'mem_no']]['ptr_mem_no'].values.tolist()
    df.loc[i, 'send_list'] = [[send_list]]
    
# 메시지 수신 회원 리스트
df['sent_list'] = None
for i in tqdm(range(len(df))) :
    sent_list = msg_df[msg_df['ptr_mem_no']==df.loc[i, 'mem_no']]['mem_no'].values.tolist()
    df.loc[i, 'sent_list'] = [[sent_list]]

100%|█████████████████████████████████████████████████████████████████████████| 124992/124992 [06:50<00:00, 304.42it/s]


In [23]:
df['send_list'] = df['send_list'].apply(lambda x : x[0])
df['send_count'] = df['send_list'].apply(lambda x : len(x))

df['sent_list'] = df['sent_list'].apply(lambda x : x[0])
df['sent_count'] = df['sent_list'].apply(lambda x : len(x))

In [24]:
df.head()

,mem_no,mem_sex,age,view_list,viewed_list,view_count,viewed_count,send_list,sent_list,send_count,sent_count
0,5,m,45,[],[],0,0,[],[],0,0
1,1793293,m,25,[],[],0,0,[],[],0,0
2,1897363,m,33,[],[],0,0,[],[],0,0
3,2,m,51,[],[],0,0,[],[],0,0
4,1688231,m,27,[],[],0,0,[],[],0,0


---

# 1. PR score model

### PR Model 전처리

In [25]:
career_key = {0 : 1, 1: 1, 2: 1, 3 : 3, 4: 4, 5: 4, 6 : 2,7 : 2, 8 : 2, 9 : 4, 10 : 4, 11 : 4, 12 : 2}
salary_key = {1 : 1,2 : 1,3: 1,4 : 2,5: 2,6 : 3,7 : 3,8 : 3, 9 : 3,10 : 4, 13 : 5, 14 : 6,15 : 7,16 :7, 17:7, 12 : 1}
possess_key = {'b' : 1, 'c' : 1, 'd' : 1, 'e' : 2, 'f' : 3, 'g' : 4, 'h' : 5 ,'m' : 6, 'i' : 7,'j' : 7, 'k' : 7, 'l' : 7, 'n' : 7}


def career_code(c) :
    if str(c).isdecimal() and int(c) in career_key.keys():
        result = career_key[int(c)]
    else :
        result = 3
    return result

def salary_code(sal) :
    if str(sal).isdecimal() and int(sal) in salary_key.keys() :
        result = salary_key[int(sal)]
    else :
        result = 1
    return result

def pos_code(pos) :
    if str(pos).isalpha() and pos in possess_key.keys() :
        result = possess_key[pos]
    else :
        result = 1
    return result

In [26]:
mate_df['mate_career'] = mate_df['mate_career'].apply(career_code)
mate_df['mate_ann_salary'] = mate_df['mate_ann_salary'].apply(salary_code)
mate_df['possess_property'] = mate_df['possess_property'].apply(pos_code)
mate_df[['mem_no','mem_loc','mate_slct','upd_cnt','photo_cnt', 'mate_career', 'mate_ann_salary', 'possess_property', 'conts_upd_cnt']]

,mem_no,mem_loc,mate_slct,upd_cnt,photo_cnt,mate_career,mate_ann_salary,possess_property,conts_upd_cnt
0,5,q,w,143,0,4,3,3,20
1,1793293,c,w,3,1,2,2,1,1
2,1897363,c,w,1,0,2,1,1,0
3,2,e,r,24,0,3,3,1,0
4,1688231,b,w,1,0,3,1,1,0
...,...,...,...,...,...,...,...,...,...
124987,1903351,b,w,1,1,1,1,1,0
124988,1893972,b,r,1,2,1,1,1,0
124989,1903539,d,w,1,1,2,2,3,0
124990,1904141,e,r,2,1,4,1,1,0


In [27]:
temp = pd.merge(df.drop(['send_list','sent_list','view_list','viewed_list'],axis=1), mate_df[['mem_no','mem_loc','mate_slct','upd_cnt','photo_cnt', 'mate_career', 'mate_ann_salary', 'possess_property', 'conts_upd_cnt']])

info = pd.merge(temp, basic_df[['mem_no','join_cnt','login_cnt','tot_stay_time']])

In [28]:
ptr_info = info.copy()
ptr_info.columns = ['ptr_' + i for i in ptr_info.columns]
ptr_info.head()

,ptr_mem_no,ptr_mem_sex,ptr_age,ptr_view_count,ptr_viewed_count,ptr_send_count,ptr_sent_count,ptr_mem_loc,ptr_mate_slct,ptr_upd_cnt,ptr_photo_cnt,ptr_mate_career,ptr_mate_ann_salary,ptr_possess_property,ptr_conts_upd_cnt,ptr_join_cnt,ptr_login_cnt,ptr_tot_stay_time
0,5,m,45,0,0,0,0,q,w,143,0,4,3,3,20,0,8512,5653082
1,1793293,m,25,0,0,0,0,c,w,3,1,2,2,1,1,0,88,5618
2,1897363,m,33,0,0,0,0,c,w,1,0,2,1,1,0,0,9,185
3,2,m,51,0,0,0,0,e,r,24,0,3,3,1,0,0,3480,960799
4,1688231,m,27,0,0,0,0,b,w,1,0,3,1,1,0,0,14,625


In [29]:
# positive response message detection
bi_temp = pd.DataFrame()
cnt = 0
temp = msg_df.copy()[['mem_no', 'ptr_mem_no']]
temp2 = temp.rename(columns={'mem_no' : 'ptr_mem_no', 'ptr_mem_no' : 'mem_no'})
for i in tqdm(range(len(temp))) :
    t = temp[(temp['mem_no']==temp2.loc[i, 'mem_no']) & (temp['ptr_mem_no']==temp2.loc[i, 'ptr_mem_no'])]
    if len(t) >= 1 :
        cnt += 1
        bi_temp = pd.concat([bi_temp, t])
        
print(cnt)
    
bi_temp['response_positive'] = 1

temp = temp.drop(bi_temp.index)
temp['response_positive'] = 0


rp = pd.concat([bi_temp, temp])
rp['response_positive'].value_counts()

100%|██████████████████████████████████████████████████████████████████████████| 19823/19823 [00:16<00:00, 1187.20it/s]

4810


0    15013
1     4810
Name: response_positive, dtype: int64

In [30]:
temp = pd.merge(rp, info)
data = pd.merge(temp, ptr_info)

In [31]:
pr_data_female = data[data['mem_sex']=='f']
pr_data_female.columns = [i.replace('ptr_','') if 'ptr_' in i or i == 'response_positive' else 'ptr_' + i for i in pr_data_female.columns]
pr_data_male = data[data['mem_sex']=='m']

data = pd.concat([pr_data_male, pr_data_female])
data.head()

,mem_no,ptr_mem_no,response_positive,mem_sex,age,view_count,viewed_count,send_count,sent_count,mem_loc,...,ptr_mate_slct,ptr_upd_cnt,ptr_photo_cnt,ptr_mate_career,ptr_mate_ann_salary,ptr_possess_property,ptr_conts_upd_cnt,ptr_join_cnt,ptr_login_cnt,ptr_tot_stay_time
4,663465,1872124,1,m,42,15,10,17,8,b,...,r,7,10,4,4,1,4,4,85,11585
5,1716776,1872124,1,m,56,31,38,39,13,h,...,r,7,10,4,4,1,4,4,85,11585
6,1504674,1872124,1,m,42,5,0,1,1,i,...,r,7,10,4,4,1,4,4,85,11585
7,1756636,1872124,0,m,44,42,31,70,6,i,...,r,7,10,4,4,1,4,4,85,11585
8,663465,1844466,1,m,42,15,10,17,8,b,...,r,11,1,2,1,1,4,0,1098,99363


In [32]:
data = data.drop(['mem_no','ptr_mem_no','mem_sex','ptr_mem_sex'], axis =1)
data['age_gap'] = (data['age'] - data['ptr_age'])

In [33]:
data['loc_like'] = data.apply(lambda x : 1 if x['mem_loc']==x['ptr_mem_loc'] else 0,axis=1)
data['marriage_like'] = data.apply(lambda x : 1 if x['mate_slct']==x['ptr_mate_slct'] else 0,axis=1)
data.drop(['mem_loc','mate_slct','ptr_mem_loc','ptr_mate_slct'],axis=1,inplace=True)

### PR Model 생성 및 검증

In [34]:
X = data.drop(['response_positive'], axis=1)
y = data['response_positive']

In [35]:
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE()
X_res, y_res = smote.fit_resample(X, y)
print('Oversampled dataset result :', Counter(y_res))

Oversampled dataset result : Counter({1: 15013, 0: 15013})


In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res,test_size=0.2)

In [37]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(random_state=0, silent=True)
model.fit(X_train, y_train)

In [38]:
pred = model.predict(X_test)
pred_proba = model.predict_proba(X_test)

from sklearn.metrics import auc, f1_score, roc_curve
fpr, tpr, threshold = roc_curve(y_test, pred_proba[:, 1])
print(f'AUC Score : {auc(fpr,tpr):.3f}')

# f1_score(y_test, pred)
print(f'f1 score : {f1_score(y_test, pred):.3f}')

AUC Score : 0.985
f1 score : 0.952


In [39]:
import joblib
joblib.dump(model, './pr_score_model.pkl')

['./pr_score_model.pkl']

In [40]:
info.columns

Index(['mem_no', 'mem_sex', 'age', 'view_count', 'viewed_count', 'send_count',
       'sent_count', 'mem_loc', 'mate_slct', 'upd_cnt', 'photo_cnt',
       'mate_career', 'mate_ann_salary', 'possess_property', 'conts_upd_cnt',
       'join_cnt', 'login_cnt', 'tot_stay_time'],
      dtype='object')

In [41]:
import numpy as np

In [42]:
def predict_positive_reply(x, top_k=50) :

    x_info = info[info['mem_no']==x]    
    x_age = x_info.age.values[0]
    x_gender = x_info.mem_sex.values[0]
    # recommendation candidate y range limit(age +- 5)
    y_range = info[(abs(x_age - info['age']) <= 5) & (info['mem_sex']!=x_gender)].mem_no.values
    # exclude viewed member
    x_hist = df[df['mem_no']==x]
    x_viewed = x_hist['view_list'].values[0]
    x_sent = x_hist['send_list'].values[0]
    y_range = [y for y in y_range if y not in x_viewed and y not in x_sent]
    y_info = info[info['mem_no'].isin(y_range)]
    
    
    if x_gender == 'f' :
        x_info.columns = ['ptr_' + c for c in x_info.columns]
        y_info['ptr_mem_no'] = x
        
    else :
        y_info.columns = ['ptr_' + c for c in y_info.columns]
        y_info['mem_no'] = x
    
    x_y_info = pd.merge(x_info, y_info)
    x_y_info.drop(['mem_no', 'ptr_mem_no', 'mem_sex', 'ptr_mem_sex'], axis=1, inplace=True)
    x_y_info['age_gap'] = x_y_info['age'] - x_y_info['ptr_age']
    
    x_y_info['loc_like'] = x_y_info.apply(lambda x : 1 if x['mem_loc']==x['ptr_mem_loc'] else 0,axis=1)
    x_y_info['marriage_like'] = x_y_info.apply(lambda x : 1 if x['mate_slct']==x['ptr_mate_slct'] else 0,axis=1)
    x_y_info.drop(['mem_loc','mate_slct','ptr_mem_loc','ptr_mate_slct'],axis=1,inplace=True)
    
    pred = model.predict_proba(x_y_info) # high probability top K
#     print(pred[:, 1])
    pred_result = pd.DataFrame(pred[:, 1], columns=['score'], index=y_range)
    pred_result = pred_result.sort_values('score', ascending=False).head(top_k)
    
    return pred_result

In [43]:
start = time.time()
predict_positive_reply(1880464)
print(f'PR score computing time : {time.time() - start : .4f}')

PR score computing time :  0.6403


---

# 2. CF score algorithm

In [44]:
def get_top_k(x, top_k=1) :
    
    
    y_range = predict_positive_reply(x).index
    
    # x에게 메시지 보낸 회원 SentTo_x
    sent_to_x = df[df['mem_no']==x].sent_list.values[0]
    # 추천 리스트[(회원번호, 점수)] 초기화
    score_list = list()
    
    print(f'my_candidate_num : {len(y_range)}') # 총 후보 수(연산 대상)
    
    for y in y_range : # 추천 후보 집단(RecommendationCandidates)의 모든 y에 대하여 반복 수행
        sent_to_y = df[df['mem_no']==y].sent_list.values[0] 
        # calculate score x_y
        score_x_y = 0 # score_x_y 초기화
        for u in sent_to_y : # y에게 메시지를 보낸 유저 집단 u에 대하여 반복 수행
            try :
                u_refrom = df[df['mem_no']==u].send_list.values[0] # u가 메시지를 보낸 유저 집합 
                x_refrom = df[df['mem_no']==x].send_list.values[0] # x가 메시지를 보낸 유저 집단
                intersection = list(set(u_refrom) & set(x_refrom)) # ReFrom_u ∩ ReFrom_x
                union = list(set(u_refrom) | set(x_refrom)) # ReFrom_u ∪ ReFrom_x
                
                if len(intersection) == 0 : # avoid zero division
                    continue
                score_x_y += len(intersection) / len(union)    # score_x_y <--- score_x_y + similarity_x_u
            except :
                pass
        score_y_x = 0 # score_y_x 초기화
        for v in sent_to_x : # x에게 메시지를 보낸 유저 집단 v에 대하여 반복 수행
            try :
                v_refrom = df[df['mem_no']==v].send_list.values[0] # v가 메시지를 보낸 유저 집합 
                y_refrom = df[df['mem_no']==y].send_list.values[0] # y가 메시지를 보낸 유저 집단
                intersection = list(set(v_refrom) & set(y_refrom)) # ReFrom_v ∩ ReFrom_y
                union = list(set(v_refrom) | set(y_refrom)) # ReFrom_v ∪ ReFrom_y
                
                if len(intersection) == 0 : # avoid zero division
                    continue
                score_y_x += len(intersection) / len(union) # score_y_x <--- score_y_x + similarity_y_v
            except :
                pass
            
        # calculate harmonic mean
        if score_x_y != 0 and score_y_x != 0 :    
            harmonic_mean_score = (2 * score_x_y * score_y_x) / (score_x_y + score_y_x)
        else :
            harmonic_mean_score = 0
        score_list.append((y, harmonic_mean_score)) # Recs <--- (y, reciprocalScore_x,y)
    
    top_k = sorted(score_list, key=lambda x : -x[1])[:top_k]
    
    return top_k # sort score and return top-k

In [45]:
start = time.time()
get_top_k(1880464)
print(f'CF score computing time : {time.time() - start : .4f}')

my_candidate_num : 50
CF score computing time :  1.8151


In [46]:
get_top_k(1880464)

my_candidate_num : 50


[(1866159, 0)]

---

# 3. Reciprocal Explanations algorithm

### 컬럼 전처리 

In [47]:
favor_food_type = {'type_1': [5, 12, 13, 15, 92, 93, 96, 97, 98],
                   'type_2': [72, 73, 74, 75, 76, 77, 82, 83],
                   'type_3': [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46],
                   'type_4': [47, 49, 50, 52, 54, 48, 57, 63, 64, 70, 71, 94, 95, 99],
                   'type_5': [6, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
                   'type_6': [8, 9, 10, 14, 17],
                   'type_7': [51, 53, 55, 56, 59, 60, 61, 62, 65],
                   'type_8': [1, 2, 3, 4, 7, 18, 80, 100],
                   'type_9': [11, 31, 58, 66, 67, 68, 69, 79, 81, 84, 85, 86, 87, 88]}

hobby_type = {'type_1': [11, 12, 18, 40, 41, 42, 43, 44],
              'type_2': [8, 15, 16, 19],
              'type_3': [1, 3, 4, 6, 13, 14, 21, 24, 25, 26, 27, 30, 32, 33, 34, 35, 39],
              'type_4': [5, 7, 17, 28, 29]}

male_char_type = {'type_1': [3, 5, 9, 13],
                  'type_2': [1, 10, 11, 17, 19],
                  'type_3': [4, 6, 8],
                  'type_4': [2, 12, 14, 15, 16, 18]}

female_char_type = {'type_1': [1, 10, 11, 17, 19],
                    'type_2': [1, 2, 11, 12, 20, 22],
                    'type_3': [4, 5, 7, 14, 9],
                    'type_4': [16, 17, 18, 19, 21, 13]}

drink_key = {'a' : 0 ,'b' : 1, 'c' : 2, 'd' : 3}
health_key = {'a' : 0, 'b' : 1, 'c' : 2, 'd' : 3}
career_key = {0 : 1, 1: 1, 2: 1, 3 : 3, 4: 4, 5: 4, 6 : 2,7 : 2, 8 : 2, 9 : 4, 10 : 4, 11 : 4, 12 : 2}
salary_key = {1 : 1,2 : 1,3: 1,4 : 2,5: 2,6 : 3,7 : 3,8 : 3, 9 : 3,10 : 4, 13 : 5, 14 : 6,15 : 7,16 :7, 17:7, 12 : 1}
possess_key = {'b' : 1, 'c' : 1, 'd' : 1, 'e' : 2, 'f' : 3, 'g' : 4, 'h' : 5 ,'m' : 6, 'i' : 7,'j' : 7, 'k' : 7, 'l' : 7, 'n' : 7}
job_key = {1: 'c', 2: 'c', 3: 'l', 4: 'l', 5: 'n', 6: 'i', 7: 'i', 8: 'i', 9: 'j', 10: 'd', 11: 'k', 12: 'k', 13: 'n',
            14: 'h', 15: 'h', 16: 'a', 17: 'a', 18: 'g', 19: 'b', 20: 'm', 21: 'o', 22: 'f', 23: 'k', 24: 'n', 25: 'e',
            26: 'e', 27: 'n', 28: 'e', 29: 'e', 0: 'e', 30: 'o', 31: 'o', 32: 'o', 33: 'j', 34: 'o', 35: 'l', 36: 'l',
            37: 'j', 38: 'n', 39: 'j', 40: 'j', 41: 'j', 42: 'j', 43: 'n', 44: 'k', 45: 'k', 46: 'c', 47: 'j', 48: 'b',
            49: 'h', 50: 'c', 51: 'c', 52: 'c', 53: 'j', 54: 'j', 55: 'j', 56: 'j', 57: 'k', 58: 'j', 59: 'i', 60: 'i',
            61: 'i', 62: 'i', 63: 'i', 64: 'i', 65: 'i', 66: 'i', 67: 'i', 68: 'g', 69: 'g', 70: 'k', 71: 'k', 72: 'h',
            73: 'g', 74: 'h', 75: 'h', 76: 'h', 77: 'h', 78: 'h', 79: 'h', 80: 'g', 81: 'a', 82: 'a', 83: 'i', 84: 'i',
            85: 'i', 86: 'a', 87: 'n', 88: 'i', 89: 'i', 90: 'i', 91: 'i', 92: 'n', 93: 'b', 94: 'b', 95: 'b', 96: 'm',
            97: 'n', 98: 'f', 99: 'f', 100: 'f', 101: 'f', 102: 'a', 103: 'f'}

def detect_type(type_li, a_type):
    for i in type_li:
        if i in a_type:
            return 1
    else:
        return 0

def drink_code(drink) :
    result = 0
    if len(str(drink)) == 0 :
        return result
    
    if str(drink)[0].isalpha() and str(drink)[0] in drink_key.keys() :
        result = drink_key[str(drink)[0]]
    return str(result)

def health_code(health) :
    result = 0
    if len(str(health)) == 0 :
        return result
    
    if str(health)[0].isalpha() and str(health)[0] in health_key.keys() :
        result = health_key[str(health)[0]]
    return str(result)

def religion_code(x) :
    if str(x).isdecimal() :
        if int(x) == 5 :
            return str(0)
        elif int(x) != 2 and int(x) != 1 :
            return str(3)
        else :
            return str(x)
    else :
        return str(0)

def job_code(my_job):
    if str(my_job).isnumeric() and int(my_job) in job_key.keys():
        result = job_key[int(my_job)]
    else:
        result = job_key[1]
    return str(result)

def career_code(c) :
    if str(c).isdecimal() and int(c) in career_key.keys():
        result = career_key[int(c)]
    else :
        result = 3
    return str(result)

def salary_code(sal) :
    if str(sal).isdecimal() and int(sal) in salary_key.keys() :
        result = salary_key[int(sal)]
    else :
        result = 1
    return str(result)

def pos_code(pos) :
    if str(pos).isalpha() and pos in possess_key.keys() :
        result = possess_key[pos]
    else :
        result = 1
    return str(result)

style_f_key = {
1:'평범함',
2:'청순함',
3:'귀여움',
4:'섹시함',
5:'날씬함',
6:'통통함',
7:'뚱뚱함',
8:'깔끔함',
9:'털털함',
10:'꽃미녀',
11:'패션왕',
12:'시크함',
13:'뇌순녀',
14:'뇌섹녀', 
}

style_m_key = {
1:'평범함',
2:'귀여움',
3:'섹시함',
4:'꽃미남',
5:'날씬함',
6:'듬직함',
7:'뚱뚱함',
8:'깔끔함',
9:'털털함',
10:'',
11:'패션왕',
12:'시크함',
13:'뇌순남',
14:'뇌섹남',
15:'근육질',
16:'건장함'
   
}

def style_code(style, sex) :
    if style.isdecimal() :
        if sex == 'm' and style in style_m_key.keys() :
            return str(style)
        elif sex == 'f' and style in style_f_key.keys() :
            return str(style)
        else :
            return '1'
    else :
        return '1'
    
def char_transform(df):
    df['mate_charc'] = df['mate_charc'].apply(lambda x: [int(x) for x in x.split(',') if x.isdecimal()])
    for i in range(1, 5) :
        df['char_type_'+str(i)] = df.apply(lambda x : detect_type(globals()['female_char_type']['type_'+str(i)],x['mate_charc']) if x['mem_sex'] == 'f' else detect_type(globals()['male_char_type']['type_'+str(i)], x['mate_charc']), axis=1)
    return df


def food_transform(df):
    df['favor_food'] = df['favor_food'].apply(lambda x: [int(i) for i in str(x).split(',') if i.isdecimal()])
    for i in range(1, 10) :
        df['food_type_'+str(i)] = df['favor_food'].apply(lambda x : detect_type(globals()['favor_food_type']['type_'+str(i)], x))
    return df


def hobby_transform(df) :
    df['mate_hobby'] = df['mate_hobby'].apply(lambda x: [int(i) for i in str(x).split(',') if i.isdecimal()])
    for i in range(1, 5) :
        df['hobby_type_'+str(i)] = df['mate_hobby'].apply(lambda x : detect_type(globals()['hobby_type']['type_'+str(i)], x))
    return df

In [48]:
explain_df = mate_df[['mem_no', 'mem_sex', 'mem_loc', 'mate_religion', 'mate_car', 'mate_job', 'mate_ann_salary',
       'mate_career', 'mate_style', 'mate_charc', 'mate_hobby', 'favor_food', 'possess_property',
        'smoke_slct', 'drink_slct','health_slct', 'mate_height', 'mate_slct']]

bins = [0, 150, 160, 170, 180, 200]
explain_df['mate_height'] = pd.cut(explain_df['mate_height'], bins=bins, labels=['under_150', '150', '160', '170', 'over_180'])

explain_df['drink_slct'] = explain_df['drink_slct'].apply(drink_code)
explain_df['smoke_slct'] = explain_df['smoke_slct'].apply(lambda x : 1 if len(str(x)) > 0 and str(x)[0] == 'c' else 0)
explain_df['health_slct'] = explain_df['health_slct'].apply(health_code)

explain_df = char_transform(explain_df)
explain_df = food_transform(explain_df)
explain_df = hobby_transform(explain_df)

explain_df['mate_religion'] = explain_df['mate_religion'].apply(religion_code)
explain_df['mate_job'] = explain_df['mate_job'].apply(job_code)
explain_df['mate_career'] = explain_df['mate_career'].apply(career_code)
explain_df['mate_car'] = explain_df['mate_car'].apply(lambda x: str(0) if x == 9 else str(1))
explain_df['mate_ann_salary'] = explain_df['mate_ann_salary'].apply(salary_code)
explain_df['possess_property'] = explain_df['possess_property'].apply(pos_code)
explain_df['mate_style'] = explain_df.apply(lambda x : style_code(x['mate_style'], x['mem_sex']), axis=1)

In [49]:
del explain_df['mate_charc'], explain_df['favor_food'], explain_df['mate_hobby'], explain_df['mem_sex']
explain_df.columns

Index(['mem_no', 'mem_loc', 'mate_religion', 'mate_car', 'mate_job',
       'mate_ann_salary', 'mate_career', 'mate_style', 'possess_property',
       'smoke_slct', 'drink_slct', 'health_slct', 'mate_height', 'mate_slct',
       'char_type_1', 'char_type_2', 'char_type_3', 'char_type_4',
       'food_type_1', 'food_type_2', 'food_type_3', 'food_type_4',
       'food_type_5', 'food_type_6', 'food_type_7', 'food_type_8',
       'food_type_9', 'hobby_type_1', 'hobby_type_2', 'hobby_type_3',
       'hobby_type_4'],
      dtype='object')

In [50]:
def get_explanation(x, y, k=3) :
    view_list = df[df['mem_no']==x].view_list.values[0]
    send_list = df[df['mem_no']==x].send_list.values[0]
    
    # m_x
    m_x = [1 if i in send_list else 0 for i in view_list]
    a_v_df = pd.DataFrame([m_x], columns=view_list, index=[x]).T

    # S_x_a_v
    for col in explain_df.columns[1:] :
        s_x_a_v = [explain_df[explain_df['mem_no']==y][col].values[0] for y in a_v_df.index]
        a_v_df[col] = s_x_a_v
        
    a_v_df = pd.get_dummies(a_v_df)

    y_a_v = pd.get_dummies(explain_df[explain_df['mem_no']==y])
    
    for y_col in y_a_v.columns :
#         print(y_col)
        if y_col in a_v_df.columns and y_a_v[y_col].values[0] != 1 :
            a_v_df.drop(y_col, inplace=True, axis=1)
#             print(f'{y_col} removed')
    a_v_df.drop([c for c in a_v_df.columns if c not in y_a_v.columns and c != x], axis=1, inplace=True)
    result = a_v_df.corr()[x].sort_values(ascending=False)
    print(f'Highest correlation features for {x} : {[result.index[1]]}, correlation value : {result.values[1]}')
    return result.index[1:k+1].tolist()

In [51]:
get_explanation(1880464, 1836325)

Highest correlation features for 1880464 : ['mate_slct_w'], correlation value : 0.17496355305594125


['mate_slct_w', 'mate_religion_1', 'food_type_1']

In [52]:
def get_reciprocal_explanation(x, y, k=1) :
    e_x_r = get_explanation(x, y, k=3)
    e_y_r = get_explanation(y, x, k=3)
    
    re_x_y = f'''{x}에게 {y}가 잘 어울리는 이유 : {e_x_r} //// {y}에게 {x}가 잘 어울리는 이유 : {e_y_r}'''
    
    return re_x_y

In [53]:
get_reciprocal_explanation(1880464, 1880995)

Highest correlation features for 1880464 : ['mate_religion_0'], correlation value : 0.22360679774997916
Highest correlation features for 1880995 : ['smoke_slct'], correlation value : nan


"1880464에게 1880995가 잘 어울리는 이유 : ['mate_religion_0', 'mem_loc_b', 'food_type_8'] //// 1880995에게 1880464가 잘 어울리는 이유 : ['smoke_slct', 'char_type_1', 'char_type_3']"

---

# 4. Final Result

In [54]:
def get_reciprocal_recommendation(x, k=1) :
    y = get_top_k(x, top_k=k)
    print(y)
    explanations = get_reciprocal_explanation(x, y[0][0])
    return y, explanations


start = time.time()

res = get_reciprocal_recommendation(1880464)

print(res)

print(f'elapsed time : {time.time() - start:.4f}')

my_candidate_num : 50
[(1866159, 0)]
Highest correlation features for 1880464 : ['hobby_type_1'], correlation value : 0.17496355305594144
Highest correlation features for 1866159 : ['smoke_slct'], correlation value : nan
([(1866159, 0)], "1880464에게 1866159가 잘 어울리는 이유 : ['hobby_type_1', 'mate_religion_1', 'mate_job_c'] //// 1866159에게 1880464가 잘 어울리는 이유 : ['smoke_slct', 'char_type_1', 'char_type_3']")
elapsed time : 2.6499


In [55]:
# get reciprocal_CF_Score
def get_top_k(x, top_k=1) :
    
    # 나이 제한 +- 5 이내에서만 검색
    x_age = df[df['mem_no']==x].age.values[0]
    x_gender = df[df['mem_no']==x].mem_sex.values[0]
    # recommendation candidate y range(age +- 5)
    y_range = df[(abs(x_age - df['age']) <= 5) & (df['mem_sex']!=x_gender)].mem_no.values
    
    # x에게 메시지 보낸 회원 SentTo_x
    sent_to_x = df[df['mem_no']==x].sent_list.values[0]
    # 추천 리스트[(회원번호, 점수)] 초기화
    score_list = list()
    
    print(f'my_candidate_num : {len(y_range)}') # 총 후보 수(연산 대상)
    
    for y in y_range : # 추천 후보 집단(RecommendationCandidates)의 모든 y에 대하여 반복 수행
        sent_to_y = df[df['mem_no']==y].sent_list.values[0] 
        # calculate score x_y
        score_x_y = 0 # score_x_y 초기화
        for u in sent_to_y : # y에게 메시지를 보낸 유저 집단 u에 대하여 반복 수행
            try :
                u_refrom = df[df['mem_no']==u].send_list.values[0] # u가 메시지를 보낸 유저 집합 
                x_refrom = df[df['mem_no']==x].send_list.values[0] # x가 메시지를 보낸 유저 집단
                intersection = list(set(u_refrom) & set(x_refrom)) # ReFrom_u ∩ ReFrom_x
                union = list(set(u_refrom) | set(x_refrom)) # ReFrom_u ∪ ReFrom_x
                
                if len(intersection) == 0 : # avoid zero division
                    continue
                score_x_y += len(intersection) / len(union)    # score_x_y <--- score_x_y + similarity_x_u
            except :
                pass
        score_y_x = 0 # score_y_x 초기화
        for v in sent_to_x : # x에게 메시지를 보낸 유저 집단 v에 대하여 반복 수행
            try :
                v_refrom = df[df['mem_no']==v].send_list.values[0] # v가 메시지를 보낸 유저 집합 
                y_refrom = df[df['mem_no']==y].send_list.values[0] # y가 메시지를 보낸 유저 집단
                intersection = list(set(v_refrom) & set(y_refrom)) # ReFrom_v ∩ ReFrom_y
                union = list(set(v_refrom) | set(y_refrom)) # ReFrom_v ∪ ReFrom_y
                
                if len(intersection) == 0 : # avoid zero division
                    continue
                score_y_x += len(intersection) / len(union) # score_y_x <--- score_y_x + similarity_y_v
            except :
                pass
            
        # calculate harmonic mean
        if score_x_y != 0 and score_y_x != 0 :    
            harmonic_mean_score = (2 * score_x_y * score_y_x) / (score_x_y + score_y_x)
        else :
            harmonic_mean_score = 0
        score_list.append((y, harmonic_mean_score)) # Recs <--- (y, reciprocalScore_x,y)
    
    top_k = sorted(score_list, key=lambda x : -x[1])[:top_k]
    
    return top_k # sort score and return top-k

def pred_positive_reply(x, y) :
    x_info = info[info['mem_no']==x]
    y_info = info[info['mem_no']==y]
    if info[info['mem_no']==x].mem_sex.values[0] == 'f' :
        x_info.columns = ['ptr_' + c for c in x_info.columns]
    else :
        y_info.columns = ['ptr_' + c for c in y_info.columns]
    x_y_info = pd.concat([y_info.reset_index(drop=True), x_info.reset_index(drop=True)],axis=1)
    x_y_info['loc_like'] = x_y_info.apply(lambda x : 1 if x['mem_loc']==x['ptr_mem_loc'] else 0, axis=1)
    x_y_info['marriage_like'] = x_y_info.apply(lambda x : 1 if x['mate_slct']==x['ptr_mate_slct'] else 0, axis=1)
    
    x_y_info.drop(['mem_no','ptr_mem_no','mem_sex','ptr_mem_sex', 'mem_loc',
                   'mate_slct', 'ptr_mem_loc', 'ptr_mate_slct'], axis=1, inplace=True)
    x_y_info['age_gap'] = x_y_info['age'] - x_y_info['ptr_age']
    pred = model.predict_proba(x_y_info)[0]
    idx = np.argmax(pred)
    return idx, pred[idx]

def get_reciprocal_score(x, a=0.3978, k=1) :
    reciprocal_score = []
    
    # 1. get CF score
    cf_time = time.time()
    cf_score = get_top_k(x, top_k=5)
    print(f'** CF score run time : {time.time() - cf_time:.3f}')
    
    # 2. get PR score
    pr_time = time.time()
    for y, cf_score in cf_score :
        pr_score = pred_positive_reply(x, y)[1]
        weighted_score = (cf_score * a) + (pr_score * (1- a)) # add weight(alpha)
        reciprocal_score.append((y, weighted_score))
    print(f'** PR score run time : {time.time() - pr_time:.3f}')
    
    # get Reciprocal Score
    top_k = sorted(reciprocal_score, key=lambda x : -x[1])
    print('candidate list :',top_k[:10])
    
    # get reciprocal explanation for x, y(candidate)
    explain_time = time.time()
    explanations = []
    for i in range(k) :
        y = top_k[i][0]
        explanations.append(get_reciprocal_explanation(x, y, k=3))
    print(f'** Reciprocal Explain run time : {time.time() - explain_time:.3f}')
    return top_k[:k], explanations

def get_reciprocal_recommendation(x) :
#     a = find_optimal_weight(x)
    rec, explanation = get_reciprocal_score(x)
    return rec, explanation

In [56]:
start = time.time()

res = get_reciprocal_recommendation(1880464)

print(res)

print(f'elapsed time : {time.time() - start:.4f}')

my_candidate_num : 9878
** CF score run time : 224.517
** PR score run time : 0.045
candidate list : [(1881040, 1.4654390165998148), (1799839, 1.3171424805882355), (1879919, 1.3080093580035448), (1393523, 1.304744280601867), (1795244, 1.2623641818747557)]
Highest correlation features for 1880464 : ['mate_religion_1'], correlation value : 0.16309808466586864
Highest correlation features for 1881040 : ['food_type_1'], correlation value : 0.21466285047213526
** Reciprocal Explain run time : 1.706
([(1881040, 1.4654390165998148)], ["1880464에게 1881040가 잘 어울리는 이유 : ['mate_religion_1', 'health_slct_2', 'mem_loc_b'] //// 1881040에게 1880464가 잘 어울리는 이유 : ['food_type_1', 'health_slct_0', 'mate_slct_r']"])
elapsed time : 226.2687
